In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
import math
import random
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
data1 = pd.read_csv("../../dataset/NASA/mw1.csv")
data1.head()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,5,7,13,0,7,8,4,0.16,4,2.0,...,24,0.11,37,47,28,9,38,21.88,25,N
1,5,21,3,1,2,24,11,0.30,4,6.0,...,23,0.24,75,86,32,16,45,7.69,37,N
2,2,5,0,0,4,8,3,0.33,4,2.0,...,9,0.19,10,17,6,9,16,30.77,9,N
3,3,5,3,0,15,8,3,0.19,4,2.0,...,12,0.09,22,33,12,14,35,48.39,16,Y
4,3,5,2,0,5,8,3,0.43,4,2.0,...,12,0.19,19,22,8,10,16,41.67,7,N


In [3]:
data1.describe()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,MULTIPLE_CONDITION_COUNT,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL
count,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,...,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000
mean,6.415020,10.280632,5.608696,0.264822,5.367589,14.482213,6.019763,0.242767,6.418972,2.287352,...,7.375494,22.355731,0.166680,46.683794,59.328063,25.849802,13.529644,39.632411,16.308933,26.849802
std,5.943992,9.023575,4.849423,1.816215,6.186523,13.941085,5.179669,0.115560,6.018355,1.057111,...,7.391900,17.301709,0.085635,38.372480,48.688016,17.206991,6.541391,29.401221,14.031681,19.824857
min,0.000000,3.000000,0.000000,0.000000,0.000000,4.000000,2.000000,0.060000,2.000000,2.000000,...,2.000000,5.000000,0.040000,4.000000,6.000000,3.000000,4.000000,5.000000,0.000000,4.000000
25%,2.000000,5.000000,2.000000,0.000000,1.000000,4.000000,3.000000,0.160000,2.000000,2.000000,...,2.000000,11.000000,0.110000,19.000000,25.000000,13.000000,9.000000,18.000000,6.250000,12.000000
50%,4.000000,7.000000,4.000000,0.000000,4.000000,8.000000,4.000000,0.220000,4.000000,2.000000,...,4.000000,16.000000,0.150000,33.000000,43.000000,21.000000,12.000000,30.000000,14.710000,20.000000
75%,9.000000,13.000000,7.000000,0.000000,7.000000,18.000000,7.000000,0.300000,8.000000,2.200000,...,9.000000,28.000000,0.200000,61.000000,80.000000,36.000000,17.000000,51.000000,23.080000,37.000000
max,34.000000,50.000000,25.000000,16.000000,42.000000,82.000000,28.000000,0.790000,36.000000,11.500000,...,48.000000,110.000000,0.690000,226.000000,303.000000,94.000000,44.000000,171.000000,65.630000,112.000000


In [4]:
data1.values.shape

(253, 38)

In [5]:
data2=data1.values
for i in range(253):
    if data2[i][37]=="Y":
        data2[i][37]=1
    else:
        data2[i][37]=0
data2[1]

array([5, 21, 3, 1, 2, 24, 11, 0.3, 4, 6.0, 2, 0.18, 32, 10, 0, 36, 1,
       47.96, 18.75, 16859.61, 0.3, 161, 0.05, 936.64, 899.18, 0.91, 10,
       12, 23, 0.24, 75, 86, 32, 16, 45, 7.69, 37, 0], dtype=object)

In [6]:
X=np.array(data2).astype(float)  

In [7]:
X[:,:37]

array([[ 5.  ,  7.  , 13.  , ..., 38.  , 21.88, 25.  ],
       [ 5.  , 21.  ,  3.  , ..., 45.  ,  7.69, 37.  ],
       [ 2.  ,  5.  ,  0.  , ..., 16.  , 30.77,  9.  ],
       ...,
       [10.  , 10.  ,  6.  , ..., 93.  , 25.61, 61.  ],
       [10.  ,  5.  ,  7.  , ..., 44.  , 21.21, 26.  ],
       [ 6.  , 25.  ,  3.  , ..., 55.  , 18.75, 39.  ]])

In [8]:
y=X[:,37]

In [9]:
X=X[:,0:37]

In [10]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
#os =  RandomOverSampler()
X_res, y_res = SMOTE().fit_resample(X, y)
X_res.shape,y_res.shape

((452, 37), (452,))

In [11]:
X_res.shape
#X_res=X_res.reshape(898,21,1)
#X_res.shape

(452, 37)

In [12]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import datetime
import csv
import time

def show_accuracy(predictLabel,Label):
    Label = np.ravel(Label).tolist()
    predictLabel = predictLabel.tolist()
    count = 0
    for i in range(len(Label)):
        if Label[i] == predictLabel[i]:
            count += 1
    return (round(count/len(Label),5))

class node_generator(object):
    def __init__(self, whiten = False):
        self.Wlist = []
        self.blist = []
        self.function_num = 0
        self.whiten = whiten

    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(x, 0)

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def linear(self, x):
        return x

    def orth(self, W):
        """
        目前看来，这个函数应该配合下一个generator函数是生成权重的
        """
        for i in range(0, W.shape[1]):
            w = np.mat(W[:,i].copy()).T
            w_sum = 0
            for j in range(i):
                wj = np.mat(W[:,j].copy()).T
                w_sum += (w.T.dot(wj))[0,0]*wj
            w -= w_sum
            w = w/np.sqrt(w.T.dot(w))
            W[:,i] = np.ravel(w)

        return W

    def generator(self, shape, times):
        for i in range(times):
            random.seed(i)
            W = 2*np.random.random(size=shape)-1
            if self.whiten == True:
                W = self.orth(W)   # 只在增强层使用
            b = 2*np.random.random() -1
            yield (W, b)

    def generator_nodes(self, data, times, batchsize, function_num):
        # 按照bls的理论，mapping layer是输入乘以不同的权重加上不同的偏差之后得到的
        # 若干组，所以，权重是一个列表，每一个元素可作为权重与输入相乘
        self.Wlist = [elem[0] for elem in self.generator((data.shape[1], batchsize), times)]
        self.blist = [elem[1] for elem in self.generator((data.shape[1], batchsize), times)]

        self.function_num = {'linear':self.linear,
                        'sigmoid': self.sigmoid,
                        'tanh':self.tanh,
                        'relu':self.relu }[function_num]  # 激活函数供不同的层选择
        # 下面就是先得到一组mapping nodes，再不断叠加，得到len(Wlist)组mapping nodes
        nodes = self.function_num(data.dot(self.Wlist[0]) + self.blist[0])
        for i in range(1, len(self.Wlist)):
            nodes = np.column_stack((nodes, self.function_num(data.dot(self.Wlist[i])+self.blist[i])))
        return nodes

    def transform(self,testdata):
        testnodes = self.function_num(testdata.dot(self.Wlist[0])+self.blist[0])
        for i in range(1,len(self.Wlist)):
            testnodes = np.column_stack((testnodes, self.function_num(testdata.dot(self.Wlist[i])+self.blist[i])))
        return testnodes

class scaler:
    def __init__(self):
        self._mean = 0
        self._std = 0
    
    def fit_transform(self,traindata):
        self._mean = traindata.mean(axis = 0)
        self._std = traindata.std(axis = 0)
        return (traindata-self._mean)/(self._std+0.001)
    
    def transform(self,testdata):
        return (testdata-self._mean)/(self._std+0.001)

class broadnet:
    def __init__(self, 
                 maptimes = 10, 
                 enhencetimes = 10,
                 map_function = 'linear',
                 enhence_function = 'linear',
                 batchsize = 'auto', 
                 reg = 0.001):
        
        self._maptimes = maptimes
        self._enhencetimes = enhencetimes
        self._batchsize = batchsize
        self._reg = reg
        self._map_function = map_function
        self._enhence_function = enhence_function
        
        self.W = 0
        self.pesuedoinverse = 0
        self.normalscaler = scaler()
        self.onehotencoder = preprocessing.OneHotEncoder(sparse = False)
        self.mapping_generator = node_generator()
        self.enhence_generator = node_generator(whiten = True)

    def fit(self,data,label,weight):
        if self._batchsize == 'auto':
            self._batchsize = data.shape[1]
        data = self.normalscaler.fit_transform(data)
        label = self.onehotencoder.fit_transform(np.mat(label).T)
        
        mappingdata = self.mapping_generator.generator_nodes(data,self._maptimes,self._batchsize,self._map_function)
        enhencedata = self.enhence_generator.generator_nodes(mappingdata,self._enhencetimes,self._batchsize,self._enhence_function)
        
        #print('number of mapping nodes {0}, number of enhence nodes {1}'.format(mappingdata.shape[1],enhencedata.shape[1]))
        #print('mapping nodes maxvalue {0} minvalue {1} '.format(round(np.max(mappingdata),5),round(np.min(mappingdata),5)))
        #print('enhence nodes maxvalue {0} minvalue {1} '.format(round(np.max(enhencedata),5),round(np.min(enhencedata),5)))
        
        inputdata = np.column_stack((mappingdata,enhencedata))
        pesuedoinverse = self.pinv(inputdata,self._reg,weight)
        self.W =  pesuedoinverse.dot(label)
        
        #print('W:', self.W)
        #print('W:', self.W.shape)  
    
    #改写伪逆矩阵算法，将权重输入
    def pinv(self,A,reg,weight):
        return np.mat(reg*np.eye(A.shape[1])+A.T.dot(weight).dot(A)).I.dot(A.T).dot(weight)
    
    def decode(self,Y_onehot):
        Y = []
        for i in range(Y_onehot.shape[0]):
            lis = np.ravel(Y_onehot[i,:]).tolist()
            Y.append(lis.index(max(lis)))
        return np.array(Y)
    
    def accuracy(self,predictlabel,label):
        label = np.ravel(label).tolist()
        predictlabel = predictlabel.tolist()
        count = 0
        for i in range(len(label)):
            if label[i] == predictlabel[i]:
                count += 1
        return (round(count/len(label),5))
        
    def predict(self,testdata):
        testdata = self.normalscaler.transform(testdata)
        test_mappingdata = self.mapping_generator.transform(testdata)
        test_enhencedata = self.enhence_generator.transform(test_mappingdata)
        
        test_inputdata = np.column_stack((test_mappingdata,test_enhencedata)) 
        #print('*predictlabel shape:',self.decode(test_inputdata.dot(self.W)).shape)
        #print('*predictlabel:', self.decode(test_inputdata.dot(self.W)))
        #print('*accuracy:',show_accuracy(self.decode(test_inputdata.dot(self.W)),testlabel))
        return self.decode(test_inputdata.dot(self.W))
    
    def addWeight(self,trainlabel):
        #对BLS设置加权，按照训练集中的label比例进行权重设置
        #求训练数据集中label为1的个数
        count=0
        for z in range(len(trainlabel)):
            #print ('个数 %d ' %z)
            #print('k_train_label:', k_train_label[z])
            if trainlabel[z]==1:
                count=count+1
        #print ('label为1的个数 %d ' %j)
        #print(count)
        #print(k_train_label.shape)
        #print(len(k_train_label))
        #权重设置 
        weight = np.zeros((len(trainlabel),len(trainlabel)))
        for i in range(len(trainlabel)):
            if trainlabel[i]==1:
                #weight[i,i]=0.618/count
                weight[i,i]=2*(len(trainlabel)-count)/len(trainlabel)
            else:
                #weight[i,i]=1/(len(k_train_label)-count)
                weight[i,i]=2*count/len(trainlabel)
        #print('Weight:', weight)
        return weight

In [13]:
#将数据集划分为训练集和测试集
#traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.2,random_state = 0)
#print(traindata.shape,trainlabel.shape,testdata.shape,testlabel.shape)
#X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
#y_train = np_utils.to_categorical(y_train)
#y_test = np_utils.to_categorical(y_test)

In [14]:
Evaluat_list=[]
for i in np.arange(0,10):
    #将数据集划分为训练集和测试集
    traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.25,random_state = i)
    bls = broadnet(maptimes = 30, 
               enhencetimes = 15,
               map_function = 'relu',
               enhence_function = 'relu',
               batchsize =100,
               reg = 0.001)

    #根据训练数据设置权重
    weight=bls.addWeight(trainlabel)

    #训练
    starttime = datetime.datetime.now()
    bls.fit(traindata,trainlabel,weight)
    endtime = datetime.datetime.now()
    trainingTime=(endtime - starttime).total_seconds()
    #print('the training time of BLS is {0} seconds'.format((endtime - starttime).total_seconds()))

    #预测
    predictlabel = bls.predict(testdata)

    #评价指标计算
    acc=accuracy_score(testlabel,predictlabel,normalize=True)
    fmeasure=f1_score(testlabel,predictlabel, average='weighted', labels=np.unique(testlabel))
    try:
        auc=roc_auc_score(testlabel,predictlabel, average='macro', sample_weight=None)
    except ValueError:
        pass
    MCC=matthews_corrcoef(testlabel,predictlabel)
    Gmeasure=geometric_mean_score(testlabel,predictlabel, average='weighted')
    recall=recall_score(testlabel, predictlabel, average='weighted')

    print('trainingTime：%f,acc：%f,fmeasure：%f,auc：%f,recall：%f,MCC：%f,Gmeasure：%f'%(trainingTime,acc,fmeasure,auc,recall,MCC,Gmeasure))
    Evaluat_tuple=(trainingTime,acc,fmeasure,auc,recall,MCC,Gmeasure)    
    Evaluat_list.append(Evaluat_tuple)

# 表头
header = ['trainTime', 'acc', 'fmeasure', 'auc','recall', 'MCC', 'Gmeasure']
with open('./data_remember/0.75WBLS_Smote_MW1_Evaluat.csv', 'w', encoding='utf-8', newline='') as file_obj:
    # 创建对象
    writer = csv.writer(file_obj)
    # 写表头
    writer.writerow(header)
    # 3.写入数据(一次性写入多行)
    writer.writerows(Evaluat_list)


    #sio.savemat('./data_remember/WBLS_Smote_CM1_test.mat',{'time':format((endtime - starttime).total_seconds()),'acc':acc,'f1':fmeasure,'auc':auc,'mcc':MCC,'Gm':Gmeasure})

trainingTime：3.402488,acc：0.911504,fmeasure：0.911017,auc：0.913793,recall：0.911504,MCC：0.836819,Gmeasure：0.913790
trainingTime：3.426042,acc：0.938053,fmeasure：0.937040,auc：0.923913,recall：0.938053,MCC：0.876143,Gmeasure：0.923805
trainingTime：3.447589,acc：0.938053,fmeasure：0.937426,auc：0.931373,recall：0.938053,MCC：0.880466,Gmeasure：0.931349
trainingTime：3.444044,acc：0.867257,fmeasure：0.866945,auc：0.880952,recall：0.867257,MCC：0.765559,Gmeasure：0.880846
trainingTime：3.445863,acc：0.902655,fmeasure：0.903159,auc：0.915385,recall：0.902655,MCC：0.822119,Gmeasure：0.915296
trainingTime：3.466049,acc：0.946903,fmeasure：0.946878,auc：0.949153,recall：0.946903,MCC：0.899152,Gmeasure：0.949150
trainingTime：3.484812,acc：0.902655,fmeasure：0.901810,auc：0.903509,recall：0.902655,MCC：0.821293,Gmeasure：0.903508
trainingTime：3.475757,acc：0.840708,fmeasure：0.836366,auc：0.834118,recall：0.840708,MCC：0.706530,Gmeasure：0.834092
trainingTime：3.463539,acc：0.902655,fmeasure：0.901810,auc：0.903509,recall：0.902655,MCC：0.821293,G